In [73]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tkinter as tk
from tkinter import Frame
from PIL import Image
from PIL import ImageTk


img = cv2.imread('000196.jpg', cv2.IMREAD_COLOR)
#img = cv2.resize(img, (520, 370))

root = tk.Tk()

import xml.etree.ElementTree as ET
from xml.etree.ElementTree import Element, dump, ElementTree

class Polygon_Proj:
    def __init__(self,file,img,root):
        self.root = object()
        self.data = file
        self.img1 = img
        self.img2 = img.copy()
        self.window = root
        self.img_list = []
    def get_xml(self):
        doc = ET.parse(self.data)
        self.root = doc.getroot()
        for labels in self.root[1][0]:
            if labels.tag == 'labels':
                label_name = [label[0].text for label in labels]
                break
        print('labels :', str(label_name).replace(",",''))

    def extract_points(self,sub_root):
        point_list = []
        for points in sub_root: 
            polygon = points.attrib
            point_list.append(polygon['points'])
        self.draw_annotation(1, point_list)
        self.draw_annotation(2, point_list)
        
    def make_coarse_annotation(self,point_list):
        point_list = []
        for points in point_list:
            even_points = points.split(';')[1::2]  #extract only even element
            result = ";".join(even_points)
            #polygon['points'] = result
            point_list.append(result)
        self.draw_annotation(2, point_list)

    def parse_tree(self):
        for sub_root in self.root:
            if sub_root.tag == 'image':
                if sub_root.attrib['name'] == '000196.jpg':
                    self.extract_points(sub_root)
                break
    def show_cv(self, img, cv2):
        img = cv2.resize(img, dsize = (0,0), fx = 0.3, fy = 0.3, interpolation = cv2.INTER_LINEAR)
        cv2.imshow('image',img)
        k = cv2.waitKey()
        cv2.destroyAllWindows()
    def draw_annotation(self, option, point_list):
        pts = []
        if option == 1:
            for points in point_list:
                points = points.split(';')
                for point in points:
                    temp = point.split(',')
                    pts.append(list(map(float, temp)))
                pts = np.array(pts, np.int32)
                pts = pts.reshape((-1,1,2))
                color = list(map(int,(np.random.randint(256,size=3))))
                self.img1 = cv2.polylines(self.img1, [pts], True, color, 3)
                pts = []
            self.img1 = cv2.resize(self.img1, dsize = (0,0), fx = 0.35, fy = 0.35, interpolation = cv2.INTER_LINEAR)
        else:
            for points in point_list:
                points = points.split(';')[1::3]
                for point in points:
                    temp = point.split(',')
                    pts.append(list(map(float, temp)))
                pts = np.array(pts, np.float32)
                pts = pts.astype(int)
                pts = pts.reshape((-1,1,2))
                color = list(map(int,(np.random.randint(256,size=3))))
                self.img2 = cv2.polylines(self.img2, [pts], True, color, 3)
                pts = []
            self.img2 = cv2.resize(self.img2, dsize = (0,0), fx = 0.35, fy = 0.35, interpolation = cv2.INTER_LINEAR)
            
    def move(self):
        img = cv2.imread('test.jpg', cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (520,350))
        cv2.imshow('image',img)
        k = cv2.waitKey()
        cv2.destroyAllWindows()

    def show_polygon(self):
        
        self.img1 = cv2.cvtColor(self.img1, cv2.COLOR_BGR2RGB)
        self.img1 = Image.fromarray(self.img1)

        self.img2 = cv2.cvtColor(self.img2, cv2.COLOR_BGR2RGB)
        self.img2 = Image.fromarray(self.img2)
        
        self.window.title("script window")
        self.window.geometry("700x810+100+100")
        button1 = tk.Button(self.window, text = 'upload', bg = 'white', fg = 'black',
                          font = ('sans 12 bold'), command = get_img).place(x = 10, y = 770)
        button2 = tk.Button(self.window, text = 'open', bg = 'white', fg = 'black',
                          font = ('sans 12 bold'), command = self.show_polygon).place(x = 200, y = 770)
        button3 = tk.Button(self.window, text = 'next', bg = 'white', fg = 'black',
                          font = ('sans 12 bold'), command = self.move).place(x = 400, y = 770)

        imgtk1 = ImageTk.PhotoImage(image=self.img1)
        imgtk2 = ImageTk.PhotoImage(image=self.img2)

        label1 = tk.Label(self.window, image=imgtk1)
        label1.pack(side='top')

        label2 = tk.Label(self.window, image=imgtk2)
        label2.pack(side='top')
        self.window.mainloop()
    def get_img(self):
        image_formats = [("JPEG","*.jpg")]
        file_path_list = askopenfilenames(filetypes=image_formats, initialdir="/", title='Please select a picture to analyze')
        for file in file_path_list:
            img = cv2.imread(file, cv2.IMREAD_COLOR)
            self.img_list.append(img)
        tk.Label(self.root, text = 'image uploaded!').place(x = 50, y = 100)
    def main_window(self):
        self.window.title("script window")
        self.window.geometry("400x300+100+100")
        button = tk.Button(self.window, text = 'image choice', bg = 'white', fg = 'black',
                          font = ('sans 12 bold'), command = get_img)
        button.place(x = 20, y = 50)
        button = tk.Button(root_window, text = 'open', bg = 'white', fg = 'black',
                          font = ('sans 12 bold'), command = self.show_polygon)
        button.place(x = 200, y = 50)
        self.root_window.mainloop()
    #cv2.destroyAllWindows()
    #tree = ElementTree(self.root)
    #tree.write('ssu.xml')

if __name__ == '__main__':
    polygon = Polygon_Proj('ssu_03.xml',img,root)
    polygon.get_xml()
    polygon.parse_tree()
    polygon.show_polygon()
    print("done")

labels : ['vehicle' 'human' 'building' 'river' 'mountain' 'sky' 'etc' 'guard_rail' 'poll' 'bush' 'forest' 'tree' 'grass' 'cobbled_road' 'dirt_road' 'packing' 'line' 'asphalt']
done


In [49]:
import os
import numpy as np
import cv2
from tkinter.filedialog import askopenfilenames
import tkinter as tk
from tkinter.ttk import Progressbar, Style, Button
from time import sleep

img_list = []
def get_path():
    root_dir = "./ssu_03/"
    for (root, dirs, files) in os.walk(root_dir):
        print("# root : " + root)
        if len(dirs) > 0:
            for dir_name in dirs:
                print("dir: " + dir_name)

        if len(files) > 0:
            for file_name in files:
                file = root_dir + file_name
                img = cv2.imread(file, cv2.IMREAD_COLOR)
                img_list.append(img)

def move():
    img = cv2.resize(img_list[60], (520,350))
    cv2.imshow('image',img)
    k = cv2.waitKey()
    cv2.destroyAllWindows()
    
def get_img():
    global img_list
    image_formats = [("JPEG","*.jpg")]
    file_path_list = askopenfilenames(filetypes=image_formats, initialdir="/", title='Please select a picture to analyze')
    for file in file_path_list:
        img = cv2.imread(file, cv2.IMREAD_COLOR)
        img_list.append(img)
    tk.Label(window, text = 'image uploaded!').place(x = 50, y = 100)

root = tk.Tk()
root.title("script window")
root.geometry("400x300+100+100")


button = tk.Button(root, text = 'image choice', bg = 'white', fg = 'black',
                  font = ('sans 12 bold'), command = get_img)
button.place(x = 20, y = 50)
button = tk.Button(root, text = 'launch', bg = 'white', fg = 'black',
                  font = ('sans 12 bold'), command = move)
button.place(x = 200, y = 50)
root.mainloop()
print('done')

NameError: name 'self' is not defined

In [23]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tkinter as tk
from tkinter import Frame
from PIL import Image
from PIL import ImageTk


img1 = cv2.imread('000196.jpg', cv2.IMREAD_COLOR)

img2 = cv2.imread('test.jpg', cv2.IMREAD_COLOR)
img1 = cv2.resize(img1, (520, 370))
img2 = cv2.resize(img2, (520, 370))

window = tk.Tk()
def move():
    
    img = cv2.imread('test.jpg', cv2.IMREAD_COLOR)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (520,350))
    cv2.imshow('image',img)
    k = cv2.waitKey()
    cv2.destroyAllWindows()

def show_annotation(img1, img2, window):
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    img1 = Image.fromarray(img1)

    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
    img2 = Image.fromarray(img2)


    window.title("script window")

    window.geometry("500x800+100+100")

    button = tk.Button(window, text = 'image choice', bg = 'white', fg = 'black',
                      font = ('sans 12 bold'), command = move)
    button.pack(side = "bottom", expand=True, fill = 'both')

    imgtk1 = ImageTk.PhotoImage(image=img1)
    imgtk2 = ImageTk.PhotoImage(image=img2)

    label1 = tk.Label(window, image=imgtk1)
    label1.pack(side='top')

    label2 = tk.Label(window, image=imgtk2)
    label2.pack(side='bottom')
    window.mainloop()

show_annotation(img1, img2, window)


In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
s = Style(window)
# add the label to the progressbar style
s.layout("LabeledProgressbar",
         [('LabeledProgressbar.trough',
           {'children': [('LabeledProgressbar.pbar',
                          {'side': 'left', 'sticky': 'ns'}),
                         ("LabeledProgressbar.label",   # label inside the bar
                          {"sticky": ""})],
           'sticky': 'nswe'})])

p = Progressbar(window, orient="horizontal", length=300,
                style="LabeledProgressbar")
p.pack()

# change the text of the progressbar, 
# the trailing spaces are here to properly center the text
s.configure("LabeledProgressbar", text="0 %      ")

def fct():
    for i in range(1, 101):
        sleep(0.01)
        p.step()
        s.configure("LabeledProgressbar", text="{0} %      ".format(i))
        window.update()
